# Imports and Setting Up

In [1]:
import json
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

ncd_file = '/Users/dianakazarian/Desktop/ClassifierData/ncd.json'

In [2]:
data = pd.read_json(ncd_file, lines=True)

# Original Data from HuffPo (NCD)

We have everything we need besides the actual body of the articles.

In [31]:
data.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [33]:
data.shape

(209527, 6)

# Web Scraping Functions

In [7]:
def scrape_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        article_text = ''
        for paragraph in soup.find_all('div', class_='primary-cli cli cli-text'):
            article_text += paragraph.get_text() + '\n'
        return article_text.strip()
    except Exception as e:
        print("Error occurred while scraping:", e)
        return None

In [36]:
executor = ThreadPoolExecutor(max_workers=10)

In [37]:
def scrape_single_url(row):
    url = row.link
    try:
        article_text = scrape_article_text(url)
        return {'link': url, 'article_text': article_text}
    except Exception as e:
        print("Error occurred while scraping:", e)
        return {'link': url, 'article_text': None}

In [44]:
subset_data['article_text'] = subset_data['link'].apply(scrape_article_text)

results = executor.map(scrape_single_url, subset_data.itertuples())

# Convert the results to a list of dictionaries and create a new DataFrame
results_list = list(results)
result_df = pd.DataFrame(results_list)

# Merge the result DataFrame with the original DataFrame based on the 'link' column
df = pd.merge(subset_data, result_df, on='link', how='left')

Error occurred while scraping: HTTPSConnectionPool(host='www.huffingtonpost.comhttps', port=443): Max retries exceeded with url: //www.outsports.com/2018/4/17/17244994/jake-bain-hunter-sigmund-gay-high-school-prom (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd4976cf100>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))


<ipython-input-44-d1388bec6058>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['article_text'] = subset_data['link'].apply(scrape_article_text)


Error occurred while scraping: HTTPSConnectionPool(host='www.huffingtonpost.comhttps', port=443): Max retries exceeded with url: //www.outsports.com/2018/4/17/17244994/jake-bain-hunter-sigmund-gay-high-school-prom (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd494032d00>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Error occurred while scraping: HTTPSConnectionPool(host='www.huffingtonpost.com', port=443): Max retries exceeded with url: /entry/matt-bomer-anything-trailer_us_5acd1cb3e4b09212968c8944 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd4e1de4f40>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))


results = executor.map(scrape_single_url, data.itertuples())
results_list = list(results)
result_df = pd.DataFrame(results_list)

In [47]:
df = df[['link', 'headline', 'category', 'short_description', 'authors', 'date', 'article_text_x']]

# A Sample Article

In [49]:
df['article_text_x'][1024]

'New York City police are seeking the public’s aid to help identify a gunman targeting homeless people.\nThe man, wearing a partial face covering, was captured on surveillance cameras shooting two sleeping homeless men in lower Manhattan early Saturday, one of them fatally. The other was shot in the arm and expected to recover, according to police.\nThe man who died was shot in the head and neck, police said. Surveillance video shows the gunman walking up to the man and firing bullets into his sleeping bag.\n“Two individuals were shot while sleeping on the streets. Not committing a crime, but sleeping on the streets,” Mayor Eric Adams said at a news conference Saturday. He called it a “total abandonment and betrayal of trust.”\nThe “video is chilling, to see a cold-blooded act of murder,” Adams said. “The case is a clear and horrific intentional act of taking the life of someone, it appears, because he was homeless,” he added.\n“All New Yorkers matter. We all matter,” Adams said.\nPoli

In [53]:
df = df[(df['article_text_x'] != '') & (df['article_text_x'].notna())]

# New dataframe with `article_text` column

In [56]:
classification_df = df

In [58]:
classification_df['class'] = df['category'].map(lambda x: 1 if x == 'POLITICS' else 0)

<ipython-input-58-bbbc27dcb031>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification_df['class'] = df['category'].map(lambda x: 1 if x == 'POLITICS' else 0)


In [70]:
classification_df.to_csv('/Users/dianakazarian/Desktop/ClassifierData/classifierdata.csv', index=False)